<a href="https://colab.research.google.com/github/ian-mcnair/ForageSnap/blob/master/TFLite_model_with_Keras_and_FastAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a TFLite model with Keras

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tf-nightly-gpu-2.0-preview
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

In [0]:
tf.__version__

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Load in Data with FastAI Process

In [0]:
from fastai.vision import *

In [0]:
classes = ['Rosemary',
           'Beauty Berry', 
           'Persimmon', 
           'Goldenrod', 
           'Poison Oak', 
           'Poison Ivy', 
           'Laurel']
# harmless_classes = ['Beauty Berry', 'Persimmon', 'Goldenrod']
# harmful_classes = ['Poison Oak', 'Poison Ivy', 'Laurel']

### Harmless Classes

In [0]:
folder = 'Rosemary' # Name of Classes below
file = '/content/drive/My Drive/Project FORSCAN/Datasets/iNaturalist Data Urls/Harmless/rosemary_urls.csv'

path = Path('/content/plants')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

download_images(path/file, 
                dest, 
                max_pics = 500, 
                max_workers= 0)

In [0]:
folder = 'Beauty Berry' # Name of Classes below
file = '/content/drive/My Drive/Project FORSCAN/Datasets/iNaturalist Data Urls/Harmless/american_beautyberry_urls.csv'

path = Path('/content/plants')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

download_images(path/file, 
                dest, 
                max_pics = 500, 
                max_workers= 0)

In [0]:
folder = 'Persimmon' # Name of Classes below
file = '/content/drive/My Drive/Project FORSCAN/Datasets/iNaturalist Data Urls/Harmless/american_persimmon_urls.csv'

path = Path('/content/plants')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

download_images(path/file, 
                dest, 
                max_pics = 500, 
                max_workers= 0)

In [0]:
folder = 'Goldenrod' # Name of Classes below
file = '/content/drive/My Drive/Project FORSCAN/Datasets/iNaturalist Data Urls/Harmless/goldenrods_urls.csv'

path = Path('/content/plants')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

download_images(path/file, 
                dest, 
                max_pics = 300, 
                max_workers= 0)

### Harmful Classes

In [0]:
folder = 'Poison Oak' # Name of classes below
file = '/content/drive/My Drive/Project FORSCAN/Datasets/iNaturalist Data Urls/Poisonous/atl_poison_oak.csv'

path = Path('/content/plants')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

download_images(path/file, 
                dest, 
                max_pics = 700, 
                max_workers= 0)

In [0]:
folder = 'Poison Ivy' # Name of classes below
file = '/content/drive/My Drive/Project FORSCAN/Datasets/iNaturalist Data Urls/Poisonous/poison_ivy_urls.csv'

path = Path('/content/plants')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

download_images(path/file, 
                dest, 
                max_pics = 700, 
                max_workers= 0)

In [0]:
folder = 'Laurel' # Name of classes below
file = '/content/drive/My Drive/Project FORSCAN/Datasets/iNaturalist Data Urls/Poisonous/laurel_fam_urls.csv'

path = Path('/content/plants')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

download_images(path/file, 
                dest, 
                max_pics = 300, 
                max_workers= 0)

## Making Sure File Paths and Images are Good

In [0]:
path = Path('/content/plants/')

In [0]:
path.ls() # This documents the path. You should see harmless and poisonous

In [0]:
path = Path('/content/plants/')
for c in classes:
  print(path/c)
  verify_images(path/c, 
                delete = True, 
                max_size = 500)
  
# path = Path('/content/plants/harmful')
# for c in harmful_classes:
#   print(path/c)
#   verify_images(path/c, 
#                 delete = True, 
#                 max_size = 500)

## Setup Input Pipeline

Download the flowers dataset.

In [0]:
base_dir = os.path.join('/content/', 'plants/')

In [0]:
base_dir

Use `ImageDataGenerator` to rescale the images.

Create the train generator and specify where the train dataset directory, image size, batch size.

Create the validation generator with similar approach as the train generator with the flow_from_directory() method.

In [0]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

In [0]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

Save the labels in a file which will be downloaded later.

In [0]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [0]:
!cat labels.txt

## Create the base model from the pre-trained convnets

Create the base model from the **MobileNet V2** model developed at Google, and pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images.

First, pick which intermediate layer of MobileNet V2 will be used for feature extraction. A common practice is to use the output of the very last layer before the flatten operation, the so-called "bottleneck layer". The reasoning here is that the following fully-connected layers will be too specialized to the task the network was trained on, and thus the features learned by these layers won't be very useful for a new task. The bottleneck features, however, retain much generality.

Let's instantiate an MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the `include_top=False` argument, we load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [0]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')



## Feature extraction
You will freeze the convolutional base created from the previous step and use that as a feature extractor, add a classifier on top of it and train the top-level classifier.

In [0]:
base_model.trainable = True

In [0]:
# # Can honestly skip runnign this but it lets you choose 
# # where you want to start training the model at

# #Let's take a look to see how many layers are in the base model
# print("Number of layers in the base model: ", len(base_model.layers))

# # Fine tune from this layer onwards
# fine_tune_at = 100

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#   layer.trainable =  False

### Add a classification head

In [0]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(7, activation='softmax') # The layer amount needs to match the classes
])

### Compile the model

You must compile the model before training it.  Since there are two classes, use a binary cross-entropy loss.

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(2e-5), # 1e-5
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

### Train the model

<!-- TODO(markdaoust): delete steps_per_epoch in TensorFlow r1.14/r2.0 -->

In [0]:
epochs = 32

history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

### Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNet V2 base model as a fixed feature extractor. 

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
# plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
# plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Convert to TFLite

Saved the model using `tf.saved_model.save` and then convert the saved model to a tf lite compatible format.

In [0]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Download the converted model and labels

In [0]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')